In [1]:
import pandas as pd
import os
import s3fs # for reading from S3FileSystem
import json # for working with JSON files 

import matplotlib.pyplot as plt

from io import StringIO

pd.set_option('max_colwidth', -1)

# Load Metadata on images

* Find out the coordinate locations for 3 images of interest
* Draw the coordinates on the image
* Show the images
* Determine if would be in the crop 

In [2]:
SAGEMAKER_PATH = r'/home/ec2-user/SageMaker'
SPLIT_PATH = os.path.join(SAGEMAKER_PATH, 'classify-streetview', 'split-train-test')

df_split = pd.read_csv(os.path.join(SPLIT_PATH, 'data-split', 'final_combined_splits.csv'))
df_split.head()

,img_id,pano_id,present_ramp,missing_ramp,obstacle,surface_prob,no_sidewalk,labels_count,count_present_ramp,count_missing_ramp,count_obstacle,count_surface_prob,count_no_sidewalk,is_test_pano,includes_rare
0,352,gmMcgskONK4C-kMOGpQfAw,False,False,False,False,True,11,0,0,0,0,11,True,False
1,510,2xZABXrvlRIsTW_lb-P-Mw,True,False,False,False,True,4,2,0,0,0,2,False,False
2,583,7Np-jziLBGYvL0jxef8n6Q,False,False,False,False,True,9,0,0,0,0,9,True,False
3,878,Rh2vW4whyJGdD6M9v47NcQ,True,False,False,False,True,13,2,0,0,0,11,False,False
4,944,WeDhagO9OpPej7YAOjAs_g,False,False,False,False,True,17,0,0,0,0,17,True,False


In [3]:
img_id_list = ['10013', '11206', '1908']
df_split.loc[df_split['img_id'].isin(img_id_list)]

,img_id,pano_id,present_ramp,missing_ramp,obstacle,surface_prob,no_sidewalk,labels_count,count_present_ramp,count_missing_ramp,count_obstacle,count_surface_prob,count_no_sidewalk,is_test_pano,includes_rare
94,10013,agVPvT3Lfz672yE7LcLiGg,True,False,False,False,False,4,4,0,0,0,0,True,False
104,11206,VbCe4cGPxSo3Yg2WMj1N2A,True,False,False,False,False,4,4,0,0,0,0,False,False
273,1908,7yGvCWd2veNF4vkmASMt9A,True,False,True,False,False,7,6,0,1,0,0,False,True


In [4]:
df_labels = pd.read_csv(os.path.join(SPLIT_PATH, 'restructure_single_labels.csv'))
df_labels.head()

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes,img_id,present_ramp,missing_ramp,obstacle,surface_prob,no_sidewalk,null,sv_image_x,sv_image_y,sv_image_y_bottom_origin,label_name
0,680_45.jpg,49558,{},3,0,"{""name"":""point"",""cx"":108,""cy"":389}","{""Present Curb Ramp"":""1\n""}",680,True,False,False,False,False,False,108,389,251,Present Curb Ramp
1,680_45.jpg,49558,{},3,1,"{""name"":""point"",""cx"":160,""cy"":389}","{""Present Curb Ramp"":""1""}",680,True,False,False,False,False,False,160,389,251,Present Curb Ramp
2,680_45.jpg,49558,{},3,2,"{""name"":""point"",""cx"":607,""cy"":453}","{""Present Curb Ramp"":""1""}",680,True,False,False,False,False,False,607,453,187,Present Curb Ramp
3,680_135.jpg,51194,{},6,0,"{""name"":""point"",""cx"":18,""cy"":475}","{""Present Curb Ramp"":""1""}",680,True,False,False,False,False,False,18,475,165,Present Curb Ramp
4,680_135.jpg,51194,{},6,1,"{""name"":""point"",""cx"":245,""cy"":413}","{""Present Curb Ramp"":""1""}",680,True,False,False,False,False,False,245,413,227,Present Curb Ramp


In [5]:
image_filename_list = ['10013_135.jpg', '11206_135.jpg', '1908_135.jpg']
df_labels.loc[df_labels['filename'].isin(image_filename_list)]

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes,img_id,present_ramp,missing_ramp,obstacle,surface_prob,no_sidewalk,null,sv_image_x,sv_image_y,sv_image_y_bottom_origin,label_name
30,1908_135.jpg,45079,{},1,0,"{""name"":""point"",""cx"":392,""cy"":433}","{""Present Curb Ramp"":""2""}",1908,True,False,False,False,False,False,392,433,207,Present Curb Ramp
917,10013_135.jpg,83337,{},1,0,"{""name"":""point"",""cx"":141,""cy"":391}","{""Present Curb Ramp"":""4""}",10013,True,False,False,False,False,False,141,391,249,Present Curb Ramp
1769,11206_135.jpg,80948,{},1,0,"{""name"":""point"",""cx"":260,""cy"":463}","{""Present Curb Ramp"":""2""}",11206,True,False,False,False,False,False,260,463,177,Present Curb Ramp


# Assume that we take it if it's in the bounds of the crop

* So do a comparison tree?

In [6]:
COOR_STRING = StringIO("""
crop_number	corner_x	corner_y	size	x_crop_left	x_crop_right	y_crop_top	y_crop_bottom
1	0	200	220	0	220	200	420
2	140	200	220	140	360	200	420
3	280	200	220	280	500	200	420
4	420	200	220	420	640	200	420
5	0	340	300	0	300	340	640
6	170	340	300	170	470	340	640
7	340	340	300	340	640	340	640
""")
df_crop_coor = pd.read_csv(COOR_STRING, sep = '\t')
df_crop_coor

,crop_number,corner_x,corner_y,size,x_crop_left,x_crop_right,y_crop_top,y_crop_bottom
0,1,0,200,220,0,220,200,420
1,2,140,200,220,140,360,200,420
2,3,280,200,220,280,500,200,420
3,4,420,200,220,420,640,200,420
4,5,0,340,300,0,300,340,640
5,6,170,340,300,170,470,340,640
6,7,340,340,300,340,640,340,640


In [7]:
df_crop_coor.to_csv('crop_coor.csv', index = False)

In [8]:
df_crop_coor.dtypes


crop_number      int64
corner_x         int64
corner_y         int64
size             int64
x_crop_left      int64
x_crop_right     int64
y_crop_top       int64
y_crop_bottom    int64
dtype: object

In [9]:
df_crop_coor['x_left'] = df_crop_coor['x_crop_left']
df_crop_coor['x_right'] = df_crop_coor['x_crop_right']
df_crop_coor['y_top'] = df_crop_coor['y_crop_bottom']
df_crop_coor['y_bottom'] = df_crop_coor['y_crop_bottom']
df_crop_coor.columns

Index(['crop_number', 'corner_x', 'corner_y', 'size', 'x_crop_left',
       'x_crop_right', 'y_crop_top', 'y_crop_bottom', 'x_left', 'x_right',
       'y_top', 'y_bottom'],
      dtype='object')

In [10]:
def is_label_in_roi(x_pt, y_pt, x_left, x_right, y_top, y_bottom):
    # Where x_pt and y_pt are the coordinate location of x and y
    # x_left and x_right are the vertical bounds
    # y_top and y_bottom are the horizontal bounds of the square
    # Note that the point (0, 0) is in the top left corner
    
    # We'll just assume the point is positive 
    if (x_left < x_pt < x_right) and (y_top < y_pt < y_bottom):
        return True
    else:
        return False

In [11]:
def get_label_roi(x_pt, y_pt, list_coors):
    result_dict = {}
    
    for index, row in df_roi.iterrows():
        result_dict[df_roi['crop_number']] = is_label_in_roi(x_pt, y_pt, df_roi['x_left'], df_roi['x_right'], df_roi['y_top'], df_roi['y_bottom'])
        
    return result_dict

# This function errors out because we are comparing values

# Try to Reshape the DF to a list

In [12]:
keep_cols = ['crop_number', 'x_crop_left',
       'x_crop_right', 'y_crop_top', 'y_crop_bottom']
list_coors = df_crop_coor[keep_cols].values.tolist()
list_coors

[[1, 0, 220, 200, 420],
 [2, 140, 360, 200, 420],
 [3, 280, 500, 200, 420],
 [4, 420, 640, 200, 420],
 [5, 0, 300, 340, 640],
 [6, 170, 470, 340, 640],
 [7, 340, 640, 340, 640]]

In [13]:
def get_label_roi(x_pt, y_pt, list_coors):
    # Where there's a list of lists
    # The sublists have the following order
    
    result_dict = {}
    
    for coor in list_coors:
        result_dict[coor[0]] = is_label_in_roi(x_pt, y_pt, coor[1], coor[2], coor[3], coor[4])
        
    return result_dict

In [14]:
%%time
get_label_roi(392, 433, list_coors) #1908

CPU times: user 10 µs, sys: 1 µs, total: 11 µs
Wall time: 14.5 µs


{1: False, 2: False, 3: False, 4: False, 5: False, 6: True, 7: True}

In [15]:
%%time
get_label_roi(141, 391, list_coors) #10013_135

CPU times: user 10 µs, sys: 1 µs, total: 11 µs
Wall time: 13.8 µs


{1: True, 2: True, 3: False, 4: False, 5: True, 6: False, 7: False}

In [16]:
%%time
get_label_roi(260, 463, list_coors) #10013_135

CPU times: user 10 µs, sys: 1e+03 ns, total: 11 µs
Wall time: 14.8 µs


{1: False, 2: False, 3: False, 4: False, 5: True, 6: True, 7: False}

# Set up to run on all points

In [17]:
list_dicts = []
list_dicts.append(get_label_roi(392, 433, list_coors)) #1908
list_dicts.append(get_label_roi(141, 391, list_coors)) #10013_135
list_dicts.append(get_label_roi(260, 463, list_coors)) #10013_135
list_dicts

[{1: False, 2: False, 3: False, 4: False, 5: False, 6: True, 7: True},
 {1: True, 2: True, 3: False, 4: False, 5: True, 6: False, 7: False},
 {1: False, 2: False, 3: False, 4: False, 5: True, 6: True, 7: False}]

In [18]:
df = pd.DataFrame(list_dicts)

In [19]:
df

,1,2,3,4,5,6,7
0,False,False,False,False,False,True,True
1,True,True,False,False,True,False,False
2,False,False,False,False,True,True,False


In [20]:
df_pivot = df.unstack()
df_pivot

1  0    False
   1    True 
   2    False
2  0    False
   1    True 
   2    False
3  0    False
   1    False
   2    False
4  0    False
   1    False
   2    False
5  0    False
   1    True 
   2    True 
6  0    True 
   1    False
   2    True 
7  0    True 
   1    False
   2    False
dtype: bool

In [21]:
list_dicts = []
list_xy = [[392, 433], [141, 391], [260, 463]]
image_names = ['1908', '10013', '11206']

for xy, image_name in zip(list_xy, image_names):
    my_dict = get_label_roi(392, 433, list_coors)
    my_dict['img_id'] = image_name
    list_dicts.append(my_dict)
list_dicts


[{1: False,
  2: False,
  3: False,
  4: False,
  5: False,
  6: True,
  7: True,
  'img_id': '1908'},
 {1: False,
  2: False,
  3: False,
  4: False,
  5: False,
  6: True,
  7: True,
  'img_id': '10013'},
 {1: False,
  2: False,
  3: False,
  4: False,
  5: False,
  6: True,
  7: True,
  'img_id': '11206'}]

In [22]:
df = pd.DataFrame(list_dicts)
df = df.set_index('img_id')
df_pivot = df.unstack()
df_pivot

   img_id
1  1908      False
   10013     False
   11206     False
2  1908      False
   10013     False
   11206     False
3  1908      False
   10013     False
   11206     False
4  1908      False
   10013     False
   11206     False
5  1908      False
   10013     False
   11206     False
6  1908      True 
   10013     True 
   11206     True 
7  1908      True 
   10013     True 
   11206     True 
dtype: bool

In [23]:
df_pivot.index

MultiIndex(levels=[[1, 2, 3, 4, 5, 6, 7], ['1908', '10013', '11206']],
           codes=[[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6], [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=[None, 'img_id'])

In [24]:
# Try to reorganize index
# https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html

# Thoughts and Comments
* How to specify the label that is included in the image
* So we get true and false - how do we say what is in that image?
* How do we figure out that there multiple in an image?
* How does Galen handle this?

Looking at this for reference: https://github.com/w210-accessibility/sidewalk-cv-assets19/blob/master/new_cities/make_new_city_csvs_sliding_window.ipynb